Clasificar las imágenes en las categorías utilizadas por los modelos originales, pero qué pasaría si hay un nuevo caso de uso y no clasifica las imágenes exactamente de la misma forma que las categorías para el modelo pre-entrenado? 

Podremos construir un nuevo modelo desde cero para nuestro caso nuevo , pero para obtener buenos resultados necesito miles de fotos con etiquetas para cuales nuestro modelo dirá si son imágenes rurales o no.

Esto se llama **transferencia de aprendizaje:** nos dará buenos resultados con mucho menos. Toma lo que un modelo aprende mientras resuelve un nuevo problema.

En un aprendizaje profundo, las capas identifican formas simples, las capas posteriores identifican patrones visuales complejos y la última capa hace predicciones, por eso las capas de un modelo pre-entrenado son útiles en nuevas aplicaciones ya que los problemas de visión por computadora involucran patrones visuales similares de bajo nivel, por lo que **reutilizaremos la mayoría del modelo ResNet pre-entrenado** y simplemente se reemplaza la capa final que es identificar características e identificar si una imagen es rural o urbana en función de los resultados de esa capa anterior.

Entrada: imágenes
ResNet Model: muchas capas, última capa: capa de predicción
Salida: Predicción en 1000 categorías

Última capa (predicción): contiene información sobre nuestro contenido fotográfico almacenado como una serie de números en un tensor unidimensional (**vector**), que se puede mostrar como una serie de puntos, cada uno de ellos se llama nodo. 

Una vez que tenemos la última capa (predicción), mantenemos el modelo pre-entrenado y agregamos una nueva capa con dos nodos para poder capturar cómo la foto es urbana o rural.

Dibujamos conexiones entre los nodos de la última capa y los nuevos nodos de la nueva predicción para establecer relaciones y cómo podría afectar nuestra medida a cómo de rural es la foto (nueva clasificación). Si se establecen muchas conexiones, realizaremos un entrenamiento para ver qué nodos de la última capa sugieren que una foto es rural o no, y lo mismo para urbana.

Paso de capitación: se permite que todas las funciones de una capa, puedan influir o conectarse a una capa de predicción (nueva), cuando esto sucede hablamos de **capa densa**.

NOTA: cuando clasificamos algo en solo dos categorías, que podríamos superar con solo un nodo en la salida, en este caso una predicción de lo urbana que sería una foto, tb sería una medida de lo urbana que es una foto si tiene un 80% de probabilidades de ser urbana, es 20% probable de ser rural, pero hemos mantenido dos nodos separados en la capa de salida, usando un nodo separado para cada categoría posible.

La capa de salida nos ayudará a hacer la transición a casos en los que queremos predecir con más de 2 categorías. Obtenemos una puntuación por cada categoría y luego aplicamos una función llamada softmax. 
La función softmax transformará las putuaciones en probabilidades para que todo lo que sea positivo se suma uno que luego podrñiamos trabajar con esas probabilidades.

Incorporamos una capa densa para importar que en esta aplicación clasifiquemos las fotos en 2 categorías (urbanas y rurales), las guardamos como clases numéricas y construimos el modelo, un **modelo secuencial (sequential () )** al que podemos agregar capas. Primero **agregamos todo el modelo ResNet 50 pre-entrenado:
add(ResNet50 ( include_top=False...**
incluimos que es **top = igual a falso**, especificamos que  excluir la capa que hace predicciones en las miles categorías utilizadas. 
Tb incorporamos un archivo que no incluya pesos para esta última capa **(weights = weights_path)**.
**pooling = 'avg'** dice que si tuviéramos canales adicionales en nuestro tensor al final de este paso, desea contenerlos en un tensor 1d tomando un promedio a través de los canales.

**my_new_model.add (Dense( num classes, activation = 'softmax'))**: agregar capas densas para hacer predicciones especificamos el número de nodos y que queremos aplicar la función softmax para poder convertirla en probabilidades

**my_new_model.layers[0].trainable = False**: le dice a las decenas que no entren en la primera capa, que es el modelo ResNet50 que ya fue pre-entrenado previamente.

- Comando de compilación: le dice al tensorflow cómo actualizar las relaciones en las conexiones densas. Cuando estamos entrenando con nuestros datos tenemos un medida de pérdida que queremos minimizar

Usamos un algoritmo llamado **descenso de gradiente estocástico** para minimizar esta pérdida (entropía):

**my_new_model.compile ( optimizer = 'sgd', loss = 'categorical_crossentropy', metrics=['accuracy'])**
 No solo trata la entropía sino que tb informa de la métrica de precisión (accuracy), que es qué fracción de las fracciones fueron correctas

Dividimos los datos sin procesar en un directorio de datos de entrenamiento y uno de datos de validación. Dentro de cada uno tenemos un subdirectorio para el urbano y otro para el rural.
Hay 2 pasos para usar el generador de datos de imagen:

1. Generamos cualquier objeto en abstracto. Le decimos que queremos aplicar la función de preprocesamiento ResNet. 

from tensoflow.python.keras.preprocesing.image import ImageDataGnerator 

data_generator = ImageDataGenerator(preprocesing_function = preprocess_input)

Luego usamos un comando de flujo desde el directorio, de forma que le decimos en qué directorio están los datos, qué tamaño de imagen queremos, cuántas imágenes a la vez (batch) y que estamos clasificando los datos en distintas categorías (class_mode = 'categorical', y tb el tamaño del lote

**date_generator.flow_from_directory(....

Hacemos esto para el directorio de entrenamiento y de validación.

Nos ajustamos al modelo, le decimos la capacitación a través de un generador capacitado para leer 12 imágenes a la vez ( 6 pasos de 12 imágenes, ya que tenemos 72 imágenes).

El generador de validación lee 20 imágenes a la vez y como tenemos 20 imágenes 

Calculando... en 6 pasos está hecho, se obtiene el 79% de los datos de entrenamiento y el 95% de correctos en los de validación.





**CÓDIGO**

In [2]:
from tensorflow.python.keras.applications import ResNet50
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Flatten, GlobalAveragePooling2D

num_classes = 2
resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

my_new_model = Sequential()
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path))
my_new_model.add(Dense(num_classes, activation='softmax'))

# Say not to train first layer (ResNet) model. It is already trained
my_new_model.layers[0].trainable = False

ModuleNotFoundError: No module named 'tensorflow'

**COMPILE MODEL**

In [ ]:
my_new_model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

**FIT MODEL**

In [ ]:
from tensorflow.python.keras.applications.resnet50 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

image_size = 224
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)


train_generator = data_generator.flow_from_directory(
        '../input/urban-and-rural-photos/rural_and_urban_photos/train',
        target_size=(image_size, image_size),
        batch_size=24,
        class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
        '../input/urban-and-rural-photos/rural_and_urban_photos/val',
        target_size=(image_size, image_size),
        class_mode='categorical')

my_new_model.fit_generator(
        train_generator,
        steps_per_epoch=3,
        validation_data=validation_generator,
        validation_steps=1)

Como steps_per_epoch = 3, quiere decir que como el generador de entrenamiento puede leer hasta 24 imágenes a la vez, y nuestro conjunto es de 72, con repetirlo 3 veces ya se completaría el conjunto total de imágenes.

Nota sobre los resultados:
La precisión de la validación impresa puede ser significativamente mejor que la precisión del entrenamiento en esta etapa. Esto puede ser desconcertante al principio.

Ocurre porque la precisión del entrenamiento se calculó en varios puntos a medida que la red mejoraba (los números en las circunvoluciones se actualizaban para hacer que el modelo fuera más preciso). La red era inexacta cuando el modelo vio las primeras imágenes de entrenamiento, ya que los pesos aún no se habían entrenado / mejorado mucho. Esos primeros resultados de entrenamiento se promediaron en la medida anterior.

Las pérdidas de validación y las medidas de precisión se calcularon después de que el modelo revisó todos los datos. Por lo tanto, la red había sido completamente entrenada cuando se calcularon estos puntajes.

Este no es un problema grave en la práctica, y tendemos a no preocuparnos por eso.

**EJERCICIOS**

El camarógrafo que filmó nuestros videos de aprendizaje profundo mencionó un problema que podemos resolver con el aprendizaje profundo.

Ofrece un servicio que escanea fotografías para almacenarlas digitalmente. Él usa una máquina que escanea rápidamente muchas fotos. Pero dependiendo de la orientación de la foto original, muchas imágenes se digitalizan de lado. Los arregla manualmente, mirando cada foto para determinar cuáles rotar.

En este ejercicio, **creará un modelo que distingue qué fotos están de lado y cuáles están en posición vertical,** de modo que una aplicación podría rotar automáticamente cada imagen si es necesario.

Si iba a vender este servicio comercialmente, podría usar un gran conjunto de datos para entrenar el modelo. Pero tendrá un gran éxito incluso con un pequeño conjunto de datos. Trabajará con un pequeño conjunto de datos de imágenes de perros, la mitad de los cuales se giran hacia los lados.

Especificar y compilar el modelo tiene el mismo aspecto que en el ejemplo que ha visto. Pero deberá realizar algunos cambios para adaptarse al modelo.

**1. ESPECIFICO EL MODELO**

In [ ]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, GlobalAveragePooling2D

num_classes = 2
resnet_weights_path = '../input/resnet50/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5'

my_new_model = Sequential()
my_new_model.add(ResNet50(include_top=False, pooling='avg', weights=resnet_weights_path))
my_new_model.add(Dense(num_classes, activation='softmax'))

# Indicate whether the first layer should be trained/changed or not.
my_new_model.layers[0].trainable = False

# Check your answer
step_1.check()

**2.COMPILO EL MODELO**

In [ ]:
my_new_model.compile(optimizer='sgd', 
                     loss='categorical_crossentropy', 
                     metrics=['accuracy'])

Eso corrió casi instantáneamente. Los modelos de aprendizaje profundo tienen la reputación de ser computacionalmente exigentes. ¿Por qué corrió eso tan rápido?

**RESPUESTA: El modelo de compilación no cambia los valores en ninguna convolución. De hecho, su modelo aún no ha recibido una discusión con datos. La compilación especifica cómo su modelo realizará actualizaciones en un paso posterior en el que reciba datos. Esa es la parte que llevará más tiempo.**

Este paso tiene 3 argumentos: optimizador, pérdida y métricas.

Qué argumento puede afectar al grado de precisión? 

**RESPUESTA:**
- El optimizador (optimizer) determina cómo determinamos los valores numéricos que componen el modelo. Por lo tanto, puede afectar el modelo resultante y las predicciones.
- La pérdida (loss) determina qué objetivo optimizamos al determinar los valores numéricos en el modelo. Por lo tanto, puede afectar el modelo resultante y las predicciones.
- La métrica (metrics) determina solo lo que imprimimos mientras se construye el modelo, pero no afecta el modelo en sí.



**3. FIT MODEL**

Tenemos 220 imágenes de entrenamiento y 217 de validación. 
El generador de entrenamiento puede leer 10 imágenes a la vez. 


In [ ]:
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator

image_size = 224
data_generator = ImageDataGenerator(preprocess_input)

train_generator = data_generator.flow_from_directory(
                                        directory='../input/dogs-gone-sideways/images/train',
                                        target_size=(image_size, image_size),
                                        batch_size=10,
                                        class_mode='categorical')

validation_generator = data_generator.flow_from_directory(
                                        directory='../input/dogs-gone-sideways/images/val',
                                        target_size=(image_size, image_size),
                                        class_mode='categorical')

# fit_stats below saves some statistics describing how model fitting went
# the key role of the following line is how it changes my_new_model by fitting to data
fit_stats = my_new_model.fit_generator(train_generator,
                                       steps_per_epoch=22,
                                       validation_data=validation_generator,
                                       validation_steps=1)

RESULTADOS: 
    Precisión: 70%
    Nivel de pérdida: 17%
    Precisión del generador de validación: 96%